# Kaggle Competition: Patfinder Pawpularity score

## 2. Inference

Import libraries

In [30]:
import pandas as pd
import torch
import torch.nn as nn
from torchvision import transforms
import matplotlib.pyplot as plt
import timm
from tqdm import tqdm
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")

Device

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


Load test data

Path names: images and model

In [8]:
path = '../input/petfinder-pawpularity-score/'
img_dir = path + 'test/'
path_model_effnetfeaturemap = '../models/effnetb0_bs64_lr00001_m09.pt'
path_model_final = '../models/xgboost_effnetb0_plusfactors.json'

Load in dataframe

In [9]:
test_data = pd.read_csv(path+'test.csv')
test_data['path'] = path + 'test/' + test_data['Id'] + '.jpg'

print('Samples test:', len(test_data))
test_data.head()

Samples test: 8


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,../input/petfinder-pawpularity-score/test/4e42...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,../input/petfinder-pawpularity-score/test/80bc...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,../input/petfinder-pawpularity-score/test/8f49...


Load in dataloader

In [10]:
img_size = 224
batch_size = 256
num_workers=0#4
pin_memory=False#True

# To make transformations of the images
transform = transforms.Compose(
                   [
                    transforms.ToTensor(),
                    transforms.Resize((img_size,img_size)),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])


class Dataset_test(torch.utils.data.Dataset):
    def __init__(self, img_dir, dataframe, img_name, transform=None):
        self.img_dir = img_dir
        self.dataframe = dataframe
        self.transform = transform
        self.img_name = img_name
        
    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, ix):
        img_id = self.dataframe[self.img_name][ix] 
        img_path = self.img_dir + img_id + '.jpg'
        X_ix = plt.imread(img_path)
        if self.transform:
            X_ix = self.transform(X_ix)
        return img_id, X_ix

dataset_test = Dataset_test(img_dir, test_data , 'Id', transform=transform)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)                

In [18]:
for i, (batch_img_id, batch_X) in enumerate(dataloader_test):
    print(f"Batch #{i+1} of size: {batch_X.shape}")

Batch #1 of size: torch.Size([8, 3, 224, 224])


Load model

In [19]:
class Net_pretrained(nn.Module):
    def __init__(self, timm_model_name, target_size, pretrained=True):
        super().__init__()
        self.model = timm.create_model(timm_model_name, pretrained=pretrained)
        self.n_features = self.model.classifier.in_features
        self.model.classifier = nn.Identity()
        self.fc = nn.Linear(self.n_features, target_size)

    def feature(self, image):
        feature = self.model(image)
        return feature

    def forward(self, image):
        feature = self.feature(image)
        output = self.fc(feature)
        return output

MODEL_NAME='tf_efficientnet_b0_ns'
model = Net_pretrained(timm_model_name=MODEL_NAME, target_size=1, pretrained=False)
model.to(device)
model.load_state_dict(torch.load(path_model_effnetfeaturemap))
model.eval()

Net_pretrained(
  (model): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_r

In [20]:
tensordeprueba = torch.load('../others/tensordeprueba.pt', map_location=torch.device(device))
feature_map = model.feature(tensordeprueba)
feature_map.shape

torch.Size([5, 1280])

Feature map from images (1280 factors)

In [24]:
y_hat_all = []
id_img_all = []
bar = tqdm(dataloader_test)
with torch.no_grad():
    for id_img, batch in bar:
        # get the inputs; data is a list of [inputs]
        X = batch
        X = X.to(device)
        # forward
        y_hat = model.feature(X)
        # accum in a list predictions
        y_hat_all.append(y_hat)
        #accum in a list identifactor of the image to not loss the treace
        id_img_all.append(id_img)

# concat tensors/lists along batch dimension
predictions = torch.cat(y_hat_all, dim=0)
id_img_all = [item for t in id_img_all for item in t]
#to numpy
predictions = predictions.detach().to("cpu").numpy()
predictions.shape

100%|██████████| 1/1 [00:00<00:00, 29.31it/s]


(8, 1280)

Insert features into test dataframe

In [50]:
df_feature_map = pd.DataFrame(data=predictions)
df_feature_map['Id'] = pd.DataFrame(id_img_all) 
df_test_stack = pd.merge(test_data, df_feature_map, on=['Id','Id'], how='left')
df_test_stack = df_test_stack.drop(columns=['Id','path'])
X_test = df_test_stack

X_test.head()

,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,1,0,1,0,0,1,1,0,0,1,...,-0.111556,-0.177450,0.291735,-0.146277,0.118518,0.235638,-0.199123,0.107482,0.096906,-0.075739
1,0,1,0,0,0,0,1,1,0,0,...,-0.034974,-0.180183,0.631597,-0.135036,0.254674,0.201008,-0.113580,0.317474,0.036562,-0.048868
2,0,0,0,1,0,1,1,1,0,1,...,-0.079093,-0.192241,0.184125,-0.157960,0.115653,0.243774,-0.132768,0.085359,0.339300,-0.103666
3,1,0,1,0,0,0,0,0,0,0,...,0.007853,-0.187072,0.281911,-0.096861,0.134717,0.203111,-0.066316,0.146190,0.061294,0.013805
4,1,1,1,0,1,1,0,1,0,1,...,-0.006322,-0.089424,0.245167,-0.155365,0.129653,0.208288,-0.074588,0.082663,0.173569,-0.072046


Load xgboost model & predict

In [54]:
model = XGBRegressor()
model.load_model(path_model_final)
predictions = model.predict(X_test)

Save submission

In [56]:
test_data['Pawpularity'] = predictions
test_data[['Id', 'Pawpularity']].to_csv('submission.csv', index=False)
test_data[['Id', 'Pawpularity']]

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,50.495754
1,43a2262d7738e3d420d453815151079e,41.764740
2,4e429cead1848a298432a0acad014c9d,54.254665
3,80bc3ccafcc51b66303c2c263aa38486,43.440643
4,8f49844c382931444e68dffbe20228f4,46.554943
5,b03f7041962238a7c9d6537e22f9b017,45.081661
6,c978013571258ed6d4637f6e8cc9d6a3,46.902557
7,e0de453c1bffc20c22b072b34b54e50f,55.250084
